<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>


<h1 align="right">KSO Tutorials #12: Analyse Zooniverse classifications</h1>
<h3 align="right">Written by @jannesgg and @vykanton</h3>
<h5 align="right">Last updated: Aug 24th, 2021</h5>

### Step 0: Import Python packages and initiate koster database

In [4]:
# Save your user name, password and Zooniverse project number.
import getpass
zoo_user = getpass.getpass('Enter your Zooniverse user')
zoo_pass = getpass.getpass('Enter your Zooniverse password')
project_n = getpass.getpass('Enter the number of the Zooniverse project')

Enter your Zooniverse user········
Enter your Zooniverse password········
Enter the number of the Zooniverse project········


In [232]:
from imp import reload
reload(t12)

<module 'utils.t12_utils' from '/Users/jurie.germishuys/Workspace/odf/koster-uw/database/koster_lab_development/utils/t12_utils.py'>

In [5]:
# Import required modules
import sys
sys.path.append('..')
import utils.t12_utils as t12

# Initiate the SQL database 
%run -i "../db_starter/starter.py" --user $zoo_user --passw $zoo_pass --project_n $project_n

Updated sites
Updated movies
Updated species
Retrieving information from subjects uploaded to Zooniverse
Updated subjects


### Step 1: Specify the Zooniverse workflow id and version of interest

*Note:  A manual export in Zooniverse is required to get the most up-to-date classifications here**

In [287]:
# Display a selectable list of workflow ids with their names and a list of versions of the workflow of interest
workflow_id, workflow_version, subj_type = t12.choose_workflows(zoo_user, zoo_pass, project_n)

Dropdown(description='Workflow id:', options=(11767, 12538, 12852, 17719, 17864, 18319), value=11767)

Dropdown(description='Workflow version:', options=(1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12…

Dropdown(description='Subject type:', index=1, options=('frame', 'clip'), value='clip')

In [331]:
# Retrieve classifications from the workflow of interest
total_df, class_df = t12.get_classifications(workflow_id.value,
                                             workflow_version.value, subj_type.value, zoo_user, zoo_pass, project_n)

Retrieving classifications from Zooniverse
Classifications retrieved from Zooniverse
Subjects retrieved from Zooniverse
Zooniverse classifications have been retrieved


### Step 2: Aggregate classifications received on the workflow of interest

In [299]:
# Specify the agreement threshold required among cit scientists
import utils.t12_utils as t12
agg_users, min_users = t12.choose_agg_parameters()

FloatSlider(value=0.8, continuous_update=False, description='Aggregation threshold:', max=1.0, readout_format=…

IntSlider(value=3, continuous_update=False, description='Min numbers of users:', max=50, min=1, style=SliderSt…

In [332]:
class_df1 = t12.aggregrate_classifications(class_df, subj_type.value, agg_users.value, min_users.value)

Aggregrating the classifications


### Step 3: Summarise the number of classifications based on the agreement specified

### Step 4: Use the subject explorer widget to get more information about specific subjects and their aggregated classifications

In [340]:
# Get all classified subjects from specified workflows
subject_df = t12.process_clips(class_df) if subj_type.value == "clip" else t13.process_frames(class_df)

t12.launch_viewer(total_df, subject_df)

ToggleButtons(button_style='success', description='Subject type:', options=('Frames', 'Clips'), value='Frames'…

Output()

In [ ]:
# END